In [1]:
from datasets import load_dataset

In [2]:
dataset = load_dataset("bhavnicksm/sentihood")

Found cached dataset json (C:/Users/conni/.cache/huggingface/datasets/bhavnicksm___json/bhavnicksm--sentihood-620acc3a4b8663dd/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51)


  0%|          | 0/3 [00:00<?, ?it/s]

Dataset({
    features: ['opinions', 'id', 'text'],
    num_rows: 2977
})

In [5]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['opinions', 'id', 'text'],
        num_rows: 2977
    })
    test: Dataset({
        features: ['opinions', 'id', 'text'],
        num_rows: 1491
    })
    validation: Dataset({
        features: ['opinions', 'id', 'text'],
        num_rows: 747
    })
})


In [6]:
# batch tokenize dataset
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\conni\.cache\huggingface\datasets\bhavnicksm___json\bhavnicksm--sentihood-620acc3a4b8663dd\0.0.0\0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51\cache-5b6f9fa34656dd00.arrow


Map:   0%|          | 0/1491 [00:00<?, ? examples/s]

Loading cached processed dataset at C:\Users\conni\.cache\huggingface\datasets\bhavnicksm___json\bhavnicksm--sentihood-620acc3a4b8663dd\0.0.0\0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51\cache-8a90ca86c518a753.arrow


In [14]:
print(tokenized_datasets)
# make all dataset splits type torch
for d in tokenized_datasets.values():
    d.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "opinions"])
    print(d.format['type'])

DatasetDict({
    train: Dataset({
        features: ['opinions', 'id', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2977
    })
    test: Dataset({
        features: ['opinions', 'id', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1491
    })
    validation: Dataset({
        features: ['opinions', 'id', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 747
    })
})
torch
torch
torch


In [16]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

print(small_train_dataset)
print(small_train_dataset.format['type'])

Loading cached shuffled indices for dataset at C:\Users\conni\.cache\huggingface\datasets\bhavnicksm___json\bhavnicksm--sentihood-620acc3a4b8663dd\0.0.0\0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51\cache-0e665416524aae0f.arrow
Loading cached shuffled indices for dataset at C:\Users\conni\.cache\huggingface\datasets\bhavnicksm___json\bhavnicksm--sentihood-620acc3a4b8663dd\0.0.0\0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51\cache-d26768867ecfb953.arrow


Dataset({
    features: ['opinions', 'id', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 100
})
torch


In [ ]:
# load pre-trained model
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

In [ ]:
from transformers import TrainingArguments
training_args = TrainingArguments("test_trainer")

In [ ]:
import numpy as np
from datasets import load_metric
metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate()